In [ ]:
from utils import *
from matplotlib.lines import Line2D
from scipy.stats import wilcoxon, ttest_rel
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LogNorm, Normalize, TwoSlopeNorm
from matplotlib.patches import ConnectionPatch

import warnings
warnings.filterwarnings('ignore')

from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

mm = .1/2.54

def set_xticklabel_5year(ax):
    ax.xaxis.set_major_locator(plt.MultipleLocator(5))
    ax.xaxis.set_minor_locator(plt.MultipleLocator(1))
    return ax

nan_h = Line2D([0], [0], color="none")

npp_loss_types = ["degradation_grass_npp", "shrink_grass_npp", "degradation_crop_npp", "abandon_crop_npp"]
npp_loss_type_colors = plt.get_cmap("Set3")(np.linspace(0, 1, 6))

npp_loss_type_colors = ["#8dd3c7", "#80b1d3", "#bc80bd", "#ffed6f"]

dic_npp_loss_type_rename = {
    'degradation_grass_npp': "$Rangeland_{D}$",
    'shrink_grass_npp': "$Rangeland_{S}$",
    'degradation_crop_npp': "$Cropland_{D}$",
    'abandon_crop_npp': "$Cropland_{S}$"
}

dic_region = {
    "Northern Africa and Western Asia": "N. Africa W. Asia",
    "Sub-Saharan Africa": "Sub-Saharan Africa",
    "Central Asia and Russian Federation": "C. Asia",
    "Eastern Asia": "E. Asia",
    "Southern Asia": "S. Asia",
    "Southeastern Asia": "S. Asia",
    "Northern America": "N. America",
    "Latin America and the Caribbean": "Latin America",
    "Western Europe": "W. Europe",
    "Eastern and South-Eastern Europe": "E. Europe",
    "Oceania and Australia": "Oceania",
}

gdf_countries = gdf_world.reset_index(drop=True).reset_index().rename(columns={"index": "idx"})
gdf_countries["regi_short"] = gdf_countries["regi_pnas"].map(dic_region)
country_to_region = gdf_countries.set_index("name_long")["regi_short"].to_dict()

In [ ]:
def get_ar_circle(radius):
    if radius>2:
        radius_clip = {3: 2.54, 4:3.53, 5: 4.49, 6: 5.52, 7: 6.52, 8: 7.49, 9: 8.52, 10: 9.49, 11: 10.52, 12: 11.49, 13: 12.49, 14: 13.51, 15: 14.49, 16: 15.52, 17: 16.46, 18: 17.49, 19: 18.49, 20: 19.47, 21: 20.49, 22: 21.49, 23: 22.49, 24: 23.49, 25: 24.43}[radius]
        gdf_circle = gpd.GeoDataFrame({}, geometry=gpd.GeoDataFrame({}, geometry=gpd.points_from_xy([0], [0])).buffer(radius_clip), crs="epsg:4326")
        da_circle = xr.DataArray(np.ones((radius * 2 + 1, radius * 2 + 1)), coords={"y": np.arange(-radius, radius + 1), "x": np.arange(-radius, radius + 1)})\
            .rio.write_crs("epsg:4326")\
            .rio.clip(gdf_circle.geometry, all_touched=True, drop=False)\
            .fillna(0).astype(np.uint8)
    else:
        da_circle = xr.DataArray([[0]], coords={"y": [0], "x": [0]}).astype(np.uint8)
    
    ar_circle = da_circle.values
    return da_circle, ar_circle

def get_ar_ring(radius):
    da_circle1, ar_circle1 = get_ar_circle(radius)
    da_circle2, ar_circle2 = get_ar_circle(radius-5)
    da_circle2 = da_circle2.reindex_like(da_circle1, fill_value=0)
    
    da_circle = da_circle1 - da_circle2
    ar_circle = da_circle.values
    return da_circle, ar_circle

In [ ]:
df_influence_distance = pd.read_csv(path_data / f"ring/ttest/influence_distance.csv", index_col=[0, 1])
df_influence_distance.columns = ["CS", "CD", "GS", "GD"]

In [ ]:
df_voronoi_ratio = pd.concat([pd.read_csv(path_data / f"PSM/voronoi/ring_ratio_{year}.csv", index_col=0) 
                              for year in range(2002, 2024)])\
                        .reset_index(drop=True)

In [ ]:
def get_df_dif_matched_all():
    df_matched_m_lst = []
    df_diff_m_lst = []
    for year in range(2002, 2024):
        for radius in range(5, 26, 5):
            df_matched_m_lst.append(pd.read_csv(path_data / f"ring/df_matched_ring_{radius}km_{year}.csv", index_col=0).assign(year=year, radius=radius))
            df_diff_m_lst.append(pd.read_csv(path_data / f"ring/df_diff_ring_{radius}km_{year}.csv", index_col=0).assign(year=year, radius=radius))
    df_diff_m_all = pd.concat(df_diff_m_lst).reset_index(drop=True)
    df_matched_m_all = pd.concat(df_matched_m_lst).reset_index(drop=True)
    df_diff_m_all["country"] = df_matched_m_all["country"]

    for luc_ in ["crop", "grass"]:
        df_crop_npp_diff_country = pd.read_csv(path_data / f"npp/{luc_}_npp_diff.csv").rename(columns={"name_long": "country"})
        df_crop_npp_diff_country = df_crop_npp_diff_country.melt(id_vars="country", var_name="year", value_name=f"{luc_}_npp_diff_country")\
            .assign(year=lambda _df: _df["year"].astype(int))

        df_matched_m_all = df_matched_m_all.merge(df_crop_npp_diff_country, on=["country", "year"], how="left")
        df_matched_m_all.loc[:, f"{luc_}_npp_change_n"] = df_matched_m_all[[f"{luc_}_npp_change_n", f"{luc_}_npp_diff_country"]].bfill(axis=1).iloc[:, 0].values
        df_matched_m_all = df_matched_m_all.drop(columns=[f"{luc_}_npp_diff_country"])
        
    df_diff_m_all = df_diff_m_all.assign(country=df_matched_m_all['country']).query("country != 'China'")
    df_matched_m_all = df_matched_m_all.query("country != 'China'")
    return df_diff_m_all, df_matched_m_all

In [ ]:
df_diff_m_all, df_matched_m_all = get_df_dif_matched_all()

In [ ]:
ring_area5 = np.pi * (5 * 1000) ** 2
ring_area10 = np.pi * (10 * 1000) ** 2 - np.pi * (5 * 1000) ** 2
ring_area15 = np.pi * (15 * 1000) ** 2 - np.pi * (10 * 1000) ** 2
ring_area20 = np.pi * (20 * 1000) ** 2 - np.pi * (15 * 1000) ** 2
ring_area25 = np.pi * (25 * 1000) ** 2 - np.pi * (20 * 1000) ** 2
dic_ring_area = {5: ring_area5, 10: ring_area10, 15: ring_area15, 20: ring_area20, 25: ring_area25}
df_ring_area = pd.DataFrame({"radius": list(dic_ring_area.keys()), "ring_area": list(dic_ring_area.values())})

In [ ]:
df_voronoi_ratio_1d = df_voronoi_ratio.melt(id_vars=["year", "idx"], value_vars=["ratio5", "ratio10", "ratio15", "ratio20", "ratio25"], var_name="radius", value_name="ratio")\
    .assign(radius=lambda _df: _df["radius"].str.slice(start=5).astype(int))

In [ ]:
df_live = df_matched_m_all.query("radius == 5")[["x", "y", "year", "idx"]].copy()
xy_lst = df_live[["x", "y"]].values

path_livestock_distribute = path_data / "FAO/FAO-GLW"
da_livestock = xr.open_dataarray(path_livestock_distribute / "GLW4-2020.D-DA.GLEAM3-ALL-LU.tif").sel(band=1)
live_num = da_livestock.sel(x=xr.DataArray(xy_lst[:, 0], dims="z"), y=xr.DataArray(xy_lst[:, 1], dims="z"), method="nearest").values
df_live = df_live\
    .assign(live_num=live_num).fillna(0)\
    .assign(live_=lambda _df: _df["live_num"]>0)

In [ ]:
df_matched_m_all = df_matched_m_all.merge(df_live[["year", "idx", "live_"]], on=["year", "idx"], how="left")

In [ ]:
df_influence_distance = pd.read_csv(path_data / f"ring/ttest/influence_distance.csv", index_col=[0, 1])
df_influence_distance.columns = ["CDS", "CPD", "GS", "GD"]

df_matched_m_all_cal_npp = df_matched_m_all[[
        "crop_cy", "crop_ly", "grass_ly", "grass_cy", 
        "crop_npp_cy", "crop_npp_ly", "grass_npp_ly", "grass_npp_cy", 
        "country", "idx", "year", "radius", "live_"
    ]].copy().assign(
        crop_change=df_matched_m_all["crop_change"] - df_matched_m_all["crop_change_n"],
        crop_npp_change=df_matched_m_all["crop_npp_change"] - df_matched_m_all["crop_npp_change_n"],
        grass_change=df_matched_m_all["grass_change"] - df_matched_m_all["grass_change_n"],
        grass_npp_change=df_matched_m_all["grass_npp_change"] - df_matched_m_all["grass_npp_change_n"]
    )\
        .reset_index(drop=True)\
        .merge(df_voronoi_ratio_1d, on=["year", "idx", "radius"], how="left")\
        .merge(df_ring_area, on="radius", how="left")\
        .merge(df_influence_distance.reset_index(), on=["country", "year"], how="left")
            
df_cal_npp_crop_d = df_matched_m_all_cal_npp.query("CPD > 0").query("radius <= CPD")
df_cal_npp_crop_s = df_matched_m_all_cal_npp.query("CDS > 0").query("radius <= CDS")
df_cal_npp_grass_d = df_matched_m_all_cal_npp.query("GD > 0").query("radius <= GD")
df_cal_npp_grass_s = df_matched_m_all_cal_npp.query("GS > 0").query("radius <= GS")

degradation_grass_df = df_cal_npp_grass_d.query("grass_npp_change<0")\
    .assign(degradation_grass_npp=lambda _df: _df["grass_cy"] * _df["grass_npp_change"] * _df["ring_area"] * _df["ratio"])\
    .groupby(["year", "country"], as_index=False)["degradation_grass_npp"].sum()\
    .set_index(["year", "country"])

shrink_grass_df = df_cal_npp_grass_s.query("grass_change<0")\
    .assign(shrink_grass_npp=lambda _df: _df["grass_change"] * _df["grass_npp_cy"] * _df["ring_area"] * _df["ratio"])\
    .groupby(["year", "country"], as_index=False)["shrink_grass_npp"].sum()\
    .set_index(["year", "country"])
    
degradation_crop_df = df_cal_npp_crop_d.query("crop_npp_change<0")\
    .assign(degradation_crop_npp=lambda _df: _df["crop_cy"] * _df["crop_npp_change"] * _df["ring_area"] * _df["ratio"])\
    .groupby(["year", "country"], as_index=False)["degradation_crop_npp"].sum()\
    .set_index(["year", "country"])

abandon_crop_df = df_cal_npp_crop_s.query("crop_change<0")\
    .assign(abandon_crop_npp=lambda _df: _df["crop_change"] * _df["crop_npp_cy"] * _df["ring_area"] * _df["ratio"])\
    .groupby(["year", "country"], as_index=False)["abandon_crop_npp"].sum()\
    .set_index(["year", "country"])
    
loss_grass = degradation_grass_df.join(shrink_grass_df, how="outer").fillna(0)
loss_crop_df = degradation_crop_df.join(abandon_crop_df, how="outer").fillna(0)
loss_df = loss_grass.join(loss_crop_df, how="outer").fillna(0) * -1
loss_df = loss_df.reset_index().assign(region=lambda _df: _df["country"].map(country_to_region)).set_index(["year", "region", "country"])